In [2]:
import geemap
import ee

In [3]:
ee.Authenticate()

Enter verification code: 4/1ASc3gC0-Tz1ceXm4SVN7yPsE_NdWFizVNXBvcJju-oiZV2T5_Hdreu0x_DY

Successfully saved authorization token.


In [4]:
ee.Initialize()

## Nasa:

In [38]:
aoi = geemap.shp_to_ee('VCS2250/Calyx_VCS_2250_project.shp')
# aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# aoi = aoi.filter(ee.Filter.eq('country_na', 'Belize'))

In [5]:
year = 2100

In [35]:
ssp585_2110 = ee.Image('IPCC/AR6/SLP/ssp585_2110').select('total_values_quantile_0_5')
ssp370_2110 = ee.Image('IPCC/AR6/SLP/ssp370_2110').select('total_values_quantile_0_5')


m4 = geemap.Map()
m4.centerObject(aoi, 9)
visParams = {
  'min': 10,
  'max': 500, #TODO: Mess with this maximum number for different maximums!
  'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000'],
}
m4.addLayer(ssp585_2110, visParams, 'SSP5-8.5 2110')
m4.addLayer(ssp370_2110, visParams, 'SSP3-7.0 2110')

# vis = ssp585_2110.visualize(min=10, max=1000, palette=['0000FF', '00FFFF', 'FFFF00', 'FF0000'])
# vis2 = ssp370_2110.visualize(min=10, max=1000, palette=['0000FF', '00FFFF', 'FFFF00', 'FF0000'])

# m4.addLayer(vis, {}, 'SSP5-8.5 2110')
# m4.addLayer(vis2, {}, 'SSP3-7.0 2110')
m4.addLayer(ee.Image().paint(aoi, 0, 2),{}, 'AOI')

m4

Map(center=[24.170135459525337, 67.69670285085533], controls=(WidgetControl(options=['position', 'transparent_…

In [45]:

year = 2110

# Load NASA SLR dataset (replace with the actual dataset ID)
ssp585_2110 = ee.Image('IPCC/AR6/SLP/ssp585_2110').select('total_values_quantile_0_5')


# Reduce region to get the nearest pixel
nearest_value_dict = ssp585_2110.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=aoi,
    scale=25000,   # Adjust to match dataset pixel size
    maxPixels=1e13
)

# Extract the value from the dictionary
value = nearest_value_dict.getInfo()['total_values_quantile_0_5']  # Replace 'slr' with actual band name

print(f"Nearest SLR value at AOI: {value}")

Nearest SLR value (m) at AOI: 1132


In [112]:
def get_nasa_slr(aoi, year, scenario):
    '''
    
    Returns SLR at cite in METERS
    '''
    
    #Default dataset initialization - will throw error if it stays this way due to invalid inputs.
    dataset = None
    
    #Get year as a string
    year_string = str(year)
    
    #get dataset
    if scenario.lower() == "ssp5-8.5":
        dataset = ee.Image('IPCC/AR6/SLP/ssp585_'+year_string).select('total_values_quantile_0_5')
    elif scenario.lower() == "ssp3-7.0":
        dataset = ee.Image('IPCC/AR6/SLP/ssp370_'+year_string).select('total_values_quantile_0_5')
    else:
        raise ValueError("Inputs must either be SSP5-8.5 or SSP3-7.0")
    
    #Reduce region to nearest pixel and extracts value from said pixel
    value_mm = dataset.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=aoi,
        scale=25000,
        maxPixels=1e13
    ).getInfo()['total_values_quantile_0_5']
    
    #Return METERS value
    return value_mm/1000

In [151]:
def get_SLR_dictionary(aoi, start_year):
    '''
    according to AOI and project start year, returns SLR data in a dictionary with 2 keys: SSP3-7.0 and SSP5-8.5. 
    The values of these keys are also dictionaries, with keys referring to the year and values referring to the SLR at that time and scenario
    return this dictionary of dictionaries.
    
    '''
    
    first_year = ((start_year // 10) + 1) * 10 #First decade after start year
    last_year = first_year + 100 #century later
    decade_years = list(range(first_year, last_year + 1, 10))
    
    slr_dict = {}
        
    for scenario in ["SSP3-7.0","SSP5-8.5"]:
        scenario_dict = {}
        for year in decade_years:
            slr_value = get_nasa_slr(aoi, year, scenario)
            scenario_dict[year] = slr_value
        slr_dict[scenario] = scenario_dict
    
    return(slr_dict)
        
    
    

In [156]:
slr_data = get_SLR_dictionary(aoi, 2015)

print(f"{'Year':>6} | {'SSP3-7.0 (m)':>12} | {'SSP5-8.5 (m)':>12}")
print("-" * 36)

# Get the years (assuming both scenarios have the same years)
years = sorted(slr_data["SSP3-7.0"].keys())

for year in years:
    ssp3 = slr_data["SSP3-7.0"][year]
    ssp5 = slr_data["SSP5-8.5"][year]
    
    # Format None as "-"
    ssp3_str = f"{ssp3:.2f}" if ssp3 is not None else "-"
    ssp5_str = f"{ssp5:.2f}" if ssp5 is not None else "-"
    
    print(f"{year:>6} | {ssp3_str:>12} | {ssp5_str:>12}")

  Year | SSP3-7.0 (m) | SSP5-8.5 (m)
------------------------------------
  2020 |         0.05 |         0.04
  2030 |         0.09 |         0.09
  2040 |         0.14 |         0.15
  2050 |         0.19 |         0.21
  2060 |         0.24 |         0.27
  2070 |         0.31 |         0.36
  2080 |         0.39 |         0.45
  2090 |         0.48 |         0.57
  2100 |         0.58 |         0.70
  2110 |         0.64 |         0.78
  2120 |         0.73 |         0.90


#look at 2020 until 2120 - give both numbers, alongside difference in meters
#CSV of every decade

In [38]:
#Forest cover from the year before the project starts - 
# if project starts november or december of year, please input the NEXT year
# Erosion before raw data
# turn geotiff into polygon, calculate hectares at every elevation, then paste in google sheet

Copernicus:

In [41]:
def get_elevation_map(aoi):
    '''
    Takes in AOI, returns DEM masked to aoi.
    '''
    copernicus_dataset = ee.ImageCollection("COPERNICUS/DEM/GLO30")
    DEM = copernicus_dataset.select('DEM')
    DEM_local = DEM.median().toFloat().clip(aoi)
    
    #returns ee.Image of area of interest with DEM data inside.
    return DEM_local

In [140]:
def get_elevation_data(aoi):
    '''
    Takes in AOI as input, returns a dictionary with keys "mean", "min", and "max"
    '''
    
    #get DEM clipped to aoi
    dem = get_elevation_map(aoi)
    
    #Use reduce region with multiple reducers to calculate mean, min, and max.
    stats = dem.reduceRegion(
        reducer=ee.Reducer.mean()
                .combine(ee.Reducer.min(), sharedInputs=True)
                .combine(ee.Reducer.max(), sharedInputs=True),
        geometry=aoi,
        scale=30,  # native resolution for GLO-30
        maxPixels=1e13
    )
    
    # GetInfo to bring values to Python
    stats_dict = stats.getInfo()
    
    # Return only mean, min, max
    return {
        'mean': stats_dict.get('DEM_mean'),
        'min': stats_dict.get('DEM_min'),
        'max': stats_dict.get('DEM_max')
    }

In [132]:
def export_DEM_geotiff(aoi, folder_name):
    '''
    Exports a GeoTIFF of elevation 
    File will be named submergence.tif
    '''
    
    dem_image = get_elevation_map(aoi)
    
    geemap.ee_export_image(
        dem_image,
        filename= folder_name+"/DEM.tif",
        #FOR USERS: change line below to scale = 50 if the DEM  geotiff fails to download.
        scale = 50,
        region = aoi.geometry()
    )

## Potential submerged area

In [17]:
def calculate_inundation_height(sedimentation, SLR):
    '''
    Input: SLR and Sedimentation, inputted as METERS PER HUNDRED YEARS.
    Output: Returns their difference, as inundation
    '''
    return (SLR - sedimentation)

In [174]:
def area_inundated_hectares(aoi, inundation_height_m):
    '''
    Returns area in hectares of area of interest where elevation < inundation height
    
    Inputs: aoi (ee.image.Image or imageCollection), inundation_height_m
    Output: percent of area inundated, as a float
    '''
    
    #get DEM
    dem_image = get_elevation_map(aoi)
    
    #Make a mask where 1 = inundated (dem.lte = less than or equal to)
    inundated = dem_image.lte(inundation_height_m)
    
    #Calculate pixel area in hectares
    pixel_area_ha = ee.Image.pixelArea().divide(10000)
    
    #
    area_img_year = pixel_area_ha.updateMask(inundated)
    
    #Area of inundated pixels:
    inundated_area = area_img_year.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e13
    )
    
    return inundated_area.getInfo().get('area')

In [178]:
def area_inundated_percent(aoi, inundation_height_m):
    '''
    takes in area of interest and inundation/submergence height and calculates percent of area below this height.
    '''
    hectares = area_inundated_hectares(aoi, inundation_height_m)
    
    pixel_area_ha = ee.Image.pixelArea().divide(10000)
    total_area = pixel_area_ha.updateMask(dem_image.mask()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e13
    ).get('area')
    
    #return percentage
    return (hectares/total_area.getInfo())*100

In [116]:
def export_submergence_geotiff(aoi, inundation_height_m):
    '''
    Exports a GeoTIFF of inundated areas to output folder. 
    File will be named submergence.tif
    '''
    
    dem_image = get_elevation_map(aoi)
    
    #Mask DEM to inundated areas, where 1 = inundated
    inundated = dem_image.lte(inundation_height_m)
    inundated_dem = dem_image.updateMask(inundated)
    
    geemap.ee_export_image(
        inundated_dem,
        filename= folder_name+"/submergence.tif",
        #FOR USERS: change line below to scale = 50 if the submergence geotiff fails to download.
        scale = 30,
        region = aoi.geometry()
    )

In [117]:
export_submergence_geotiff(aoi, 0.724, 'test_output')

Generating URL ...
Please wait ...
Data downloaded to C:\Users\ajfar\Calyx\test_output\submergence.tif


# Visualization:

In [161]:
#Submergence visualization:

dem = get_elevation_map (aoi)
inundation_mask = dem.lte(0.64-0.06)
inundated_dem = dem.updateMask(inundation_mask)
map_map = geemap.Map()
map_map.centerObject(aoi, 8)

dem_stats = DEM_global.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    bestEffort=True,
    maxPixels=1e12
)
dem_stats_info = dem_stats.getInfo()

dem_min = dem_stats_info['DEM_min']
dem_max = dem_stats_info['DEM_max']
elevationVis = {
  'min': dem_min,
  'max': dem_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff'],
}
map_map.addLayer(ee.Image().paint(aoi, 0, 2),{}, 'AOI')
map_map.addLayer(dem, elevationVis, 'DEM')
map_map.addLayer(inundated_dem, {'palette': 'red'}, 'inundated')


map_map

20.551938063509507 (m)


In [183]:
print("Percent of PA inundated at 0.9m SLR: " + str(area_inundated_percent(aoi, 0.84)) + " %")
print("Percent of PA inundated at 0.78m SLR: " + str(area_inundated_percent(aoi, 0.72)) + " %")
print("Percent of PA inundated at 100000m SLR: " + str(area_inundated_percent(aoi, 100000)) + " %")
print("Percent of PA inundated at 0m submergence (elevation <0): " + str(area_inundated_percent(aoi, 0)) + " %")

Percent of PA inundated at 0.9m SLR: 29.326791078625387 %
Percent of PA inundated at 0.78m SLR: 28.270997443383077 %
Percent of PA inundated at 100000m SLR: 100.0 %
Percent of PA inundated at 0m submergence (elevation <0): 25.95588320286053 %


In [182]:
#DEM visualization:

copernicus_dataset = ee.ImageCollection("COPERNICUS/DEM/GLO30")
DEM = copernicus_dataset.select('DEM')
DEM_global = DEM.median().toFloat().clip(aoi)

dem_stats = DEM_global.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    bestEffort=True,
    maxPixels=1e12
)
dem_stats_info = dem_stats.getInfo()

dem_min = dem_stats_info['DEM_min']
print(dem_min)
dem_max = dem_stats_info['DEM_max']

m7 = geemap.Map()
m7.centerObject(aoi, 8)
elevationVis = {
  'min': dem_min,
  'max': dem_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff'],
}
m7.addLayer(ee.Image().paint(aoi, 0, 2),{}, 'AOI')
m7.addLayer(DEM_global, elevationVis, 'DEM')

m7

-6.186857223510742


Map(center=[24.170135459525337, 67.69670285085533], controls=(WidgetControl(options=['position', 'transparent_…